<div style="direction:rtl;line-height:300%;">
	<font face="XB Zar" size=5>
		<div align=center>
			<font face="IranNastaliq" size=30>
				<p></p>
				<p></p>
به نام خدا
				<p></p>
			</font>
			<p></p>
			<font color=blue>
پروژه ی filler
            </font>
			<br />
			<br />
دی ماه 97
		</div>
		<hr/>
		<font color=red size=6>
			<br />
			<div align=center>
while loop
            </div>        
		</font>
		<br />
		<div align=center>
 مهدی غزنوی
        </div>
		<hr />
		<style type="text/css" scoped>
        p{
        border: 1px solid #a2a9b1;background-color: #f8f9fa;display: inline-block;
        };
        </style>
	</font>
</div>

<div style="direction:rtl;line-height:300%;">
    <font face="XB Zar" size=5>
        یکی از چابش های اصلی در tensorflow تکرار یک کار است. دلیل چالشی بودن موضوع این است که tensorflow با ساختن یک گراف از اعمال مورد نظر ما و اجرای آن کار می کند. مشکلی که در اینجا به وجود می آید این است که ممکن بخواهیم تعداد اجرای یک عمل بر حسب ورودی متفاوت باشد؛ در اینصورت گراف operationهای ما برای ورودی های مختلف باید فرق کند.<br/>
        راه حل tensorflow برای این موضوع استفاده از tf.while_loop است. به صورت خلاصه tf.while_loop با گرفتن یک body و یک condition درست مانند یک loop عادی تا زمانی که cond برقرار است body را انجام می دهد 
    </font> 
</div>

In [1]:
def while_loop(cond,
               body,
               loop_vars,
               shape_invariants=None,
               parallel_iterations=10,
               back_prop=True,
               swap_memory=False,
               name=None,
               maximum_iterations=None,
               return_same_structure=False):
    pass

<div style="direction:rtl;line-height:300%;">
    <font face="XB Zar" size=5>
        body و cond هر دو توابعی هستند که loop_vars به عنوان ورودی به آنها داده می شود. در تابع بالا loop_vars ورودی های اولیه به body و condition است. در مراحل بعدی اجرای حلقه نیز ورودی ها باید ساختاری مشابه loop_vars داشته باشند. به عبارت دیگر خروجی های body نیز باید هم ساختار loop_vars باشد.<br/>
    </font> 
</div>

<div style="direction:rtl;line-height:300%;">
    <font face="XB Zar" size=5>
        tensorflow این قابلیت را به ما داده است که در body خروجی هایی با ساختار متفاوت با loop_vars داشته باشیم. به این منظور می توانیم از shape_varients استفاده کنیم. در shape_varients باید برای هرکدام از عاصر loop_vars یک shape مشخص کنیم. این shape متفاوت کلی تر از shape عضو متناظر در loop_vars باشد. مثال زیر به درک بهتر کاربرد این loop_varients کمک می کند.<br/>
        برابر بودن خروجی کل while_loop با ورودی آن را نیز می توان با return_same_structure کنترل کرد.
    </font> 
</div>

In [3]:
import tensorflow as tf
i0 = tf.constant(0)
m0 = tf.ones([2, 2])
c = lambda i, m: i < 10 #condition
b = lambda i, m: [i+1, tf.concat([m, m], axis=0)] #body
tf.while_loop(
    c, b, loop_vars=[i0, m0],
    shape_invariants=[i0.get_shape(), tf.TensorShape([None, 2])])

[<tf.Tensor 'while/Exit:0' shape=() dtype=int32>,
 <tf.Tensor 'while/Exit_1:0' shape=(?, 2) dtype=float32>]

<div style="direction:rtl;line-height:300%;">
    <font face="XB Zar" size=5>
        نحوه ی کار while_loop به این صورت است که ابتدا گراف body و condition را تشکیل می دهد و سپس خروجی ها را به ورودی ها وصل می کند. اجرای گراف و تولید خروجی ها ممکن است قابلیت اجرا به صورت موازی را داشته باشد. در ورودی while_loop می توان حداکثر تعداد پردازش های موازی به هنوان parallel_iterations مشخص کرد.
    </font> 
</div>

<div style="direction:rtl;line-height:300%;">
    <font face="XB Zar" size=5>
        گاهی اوقات ممکن است یک treshhold برای تعداد عملیات هایمان داشته باشیم. برای مثال با وجود اینکه پایان جملات را با EOS مشخص می کنیم قالبا یک طول بیشینه نیز برای آن در نظر می گیریم که در صورتی که برنامه بیش از آن مقدار token تولید کرد جمله را خاتمه می دهیم.<br/>
        این treshhold را می توانیم در maximum_iterations تعیین کنیم.
    </font> 
</div>

<div style="direction:rtl;line-height:300%;">
    <font face="XB Zar" size=5>
        <b>نکته</b>: توابع body و cond تنها یک بار و در زمان ساختن گراف صدا زده می شوند.
    </font> 
</div>

In [7]:
import tensorflow as tf

n = 10000
x = tf.constant(list(range(n)))
c = lambda i, x: i < n
b = lambda i, x: (tf.Print(i + 1, [i]), tf.Print(x + 1, [i], "x:"))
i, out = tf.while_loop(c, b, (0, x))
with tf.Session() as sess:
    print(sess.run(i))  # prints [0] ... [9999]

    # The following line may increment the counter and x in parallel.
    # The counter thread may get ahead of the other thread, but not the
    # other way around. So you may see things like
    # [9996] x:[9987]
    # meaning that the counter thread is on iteration 9996,
    # while the other thread is on iteration 9987
    print(sess.run(out).shape)

10000
(10000,)


<div style="direction:rtl;line-height:300%;">
    <font face="XB Zar" size=5>
        اکنون بیایید مثال بالا از صفحه ی while_loop در سایت tensorflow را بررسی کنیم.<br/>
        چون خروجی x و i ربطی به هم ندارند پس با اجرای (sess.run(i تنها قسمت اول b اجرا می شود و درنتیجه خروجی به صورت:<br/>[0]<br/> ... <br/>[9999]<br/>(sess.run(x نیز به صورت مشابه عمل می کند. 
    </font> 
</div>